In [1]:
from gnncloudmanufacturing.data import read_fatahi_dataset
from gnncloudmanufacturing.random_solution import random_solve
from gnncloudmanufacturing.validation import total_cost_from_graph, check_feasibility, total_cost_from_gamma
from gnncloudmanufacturing.utils import delta_from_gamma, graph_from_problem, gamma_from_target, delta_from_gamma
from gnncloudmanufacturing.graph_model import GNN

import numpy as np
from tqdm.auto import trange, tqdm
from time import time
import pandas as pd

In [2]:
dataset = read_fatahi_dataset('../data/fatahi.xlsx')
for problem in dataset:
    print(f'Problem: {problem["name"]}')

  0%|          | 0/18 [00:00<?, ?it/s]

Problem: 5,10,10-1
Problem: 5,10,10-2
Problem: 5,10,10-3
Problem: 10,10,10-1
Problem: 10,10,10-2
Problem: 10,10,10-3
Problem: 5,10,20-1
Problem: 5,10,20-2
Problem: 5,10,20-3
Problem: 5,20,10-1
Problem: 5,20,10-2
Problem: 5,20,10-3
Problem: 5,20,20-1
Problem: 5,20,20-2
Problem: 5,20,20-3
Problem: 5,5,5-1
Problem: 5,5,5-2
Problem: 5,5,5-3


In [3]:
model = GNN.load_from_checkpoint(
    checkpoint_path="gnn_ver0.ckpt",
    ins_dim=1,
    ino_dim=12,
    out_dim=16,
    n_layers=1,
    lr=0.001,
)

In [4]:
problem_name = []
total_cost = []
comp_time = []
for problem in tqdm(dataset):
    if problem['n_operations'] != 10:
        continue
    start = time()
    total = np.inf
    for i in range(10):
        graph = graph_from_problem(problem)
        pred = model.predict(graph)
        gamma = gamma_from_target(pred, graph, problem)
        delta = delta_from_gamma(problem, gamma)
        check_feasibility(gamma, delta, problem)
        _total = total_cost_from_gamma(problem, gamma, delta).item()
        if total > _total:
            total = _total
    total_cost.append(total)
    problem_name.append(problem['name'])
    comp_time.append(time() - start)

  0%|          | 0/18 [00:00<?, ?it/s]

In [7]:
pd.DataFrame({'problem_name': problem_name, 'total_cost': total_cost, 'comp_time': comp_time}).round(2)

,problem_name,total_cost,comp_time
0,"5,10,10-1",13024.02,0.93
1,"5,10,10-2",19176.68,0.77
2,"5,10,10-3",24126.72,0.79
3,"10,10,10-1",38551.73,1.09
4,"10,10,10-2",42619.50,1.12
5,"10,10,10-3",44024.86,1.17
6,"5,10,20-1",13038.87,1.67
7,"5,10,20-2",21067.38,1.85
8,"5,10,20-3",21002.83,1.89
